In [1]:
import numpy as np
import pandas as pd
import pdb

In [3]:
action_space = [(0,4),#0
 (1,4),#1
 (1,5),#2
 (2,5),#3
 (2,6),#4
 (3,6),#5
 (3,7),#6
 (4,0),#7
 (4,1),#8
 (4,8),#9
 (4,9),#10
 (5,1),#11
 (5,2),#12
 (5,9),#13
 (5,10),#14
 (6,2),#15
 (6,3),#16
 (6,10),#17
 (6,11),#18
 (7,3),#19
 (7,11),#20
 (8,4),#21
 (8,12),#22
 (9,4),#23
 (9,5),#24
 (9,12),#25
 (9,13),#26
 (10,5),#27
 (10,6),#28
 (10,13),#29
 (10,14),#30
 (11,6),#31
 (11,7),#32
 (11,14),#33
 (11,15),#34
 (12,8),#35
 (12,9),#36
 (12,16),#37
 (12,17),#38
 (13,9),#39
 (13,10),#40
 (13,17),#41
 (13,18),#42
 (14,10),#43
 (14,11),#44
 (14,18),#45
 (14,19),#46
 (15,11),#47
 (15,19),#48
 (16, 12),#49
 (16, 20),#50
 (17, 12),#51
 (17, 13),#52
 (17, 20),#53
 (17, 21),#54
 (18, 13),#55
 (18, 14),#56
 (18, 21),#57
 (18, 22),#58
 (19, 14),#59
 (19, 15),#60
 (19, 22),#61
 (19, 23),#62
 (20, 16),#63
 (20, 17),#64
 (20, 24),#65
 (20, 25),#66
 (21, 17),#67
 (21, 18),#68
 (21, 25),#69
 (21, 26),#70
 (22, 18),#71
 (22, 19),#72
 (22, 26),#73
 (22, 27),#74
 (23, 19),#75
 (23, 27),#76
 (24, 20),#77
 (24, 28),#78
 (25, 20),#79
 (25, 21),#80
 (25, 28),#81
 (25, 29),#82
 (26, 21),#83
 (26, 22),#84
 (26, 29),#85
 (26, 30),#86
 (27, 22),#87
 (27, 23),#88
 (27, 30),#89
 (27, 31),#90
 (28, 24),#91
 (28, 25),#92
 (29, 25),#93
 (29, 26),#94
 (30, 26),#95
 (30, 27),#96
 (31, 27)]#97

In [4]:
numbers = np.arange(1,99,1)

In [5]:
action_dict = dict(zip(numbers,action_space))

In [6]:
action_dict

{1: (0, 4),
 2: (0, 5),
 3: (1, 5),
 4: (1, 6),
 5: (2, 6),
 6: (2, 7),
 7: (3, 7),
 8: (4, 0),
 9: (4, 8),
 10: (5, 0),
 11: (5, 1),
 12: (5, 8),
 13: (5, 9),
 14: (6, 1),
 15: (6, 2),
 16: (6, 9),
 17: (6, 10),
 18: (7, 2),
 19: (7, 3),
 20: (7, 10),
 21: (7, 11),
 22: (8, 4),
 23: (8, 5),
 24: (8, 12),
 25: (8, 13),
 26: (9, 5),
 27: (9, 6),
 28: (9, 13),
 29: (9, 14),
 30: (10, 6),
 31: (10, 7),
 32: (10, 14),
 33: (10, 15),
 34: (11, 7),
 35: (11, 15),
 36: (12, 8),
 37: (12, 16),
 38: (13, 8),
 39: (13, 9),
 40: (13, 16),
 41: (13, 17),
 42: (14, 9),
 43: (14, 10),
 44: (14, 17),
 45: (14, 18),
 46: (15, 10),
 47: (15, 11),
 48: (15, 18),
 49: (15, 19),
 50: (16, 12),
 51: (16, 13),
 52: (16, 20),
 53: (16, 21),
 54: (17, 13),
 55: (17, 14),
 56: (17, 21),
 57: (17, 22),
 58: (18, 14),
 59: (18, 15),
 60: (18, 22),
 61: (18, 23),
 62: (19, 15),
 63: (19, 23),
 64: (20, 16),
 65: (20, 24),
 66: (21, 16),
 67: (21, 17),
 68: (21, 24),
 69: (21, 25),
 70: (22, 17),
 71: (22, 18),
 7

In [7]:
initial_state = [1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]

In [8]:
zeros = np.zeros((1,98))

In [9]:
Q = np.zeros((1,98))

In [10]:
Q

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.]])

In [11]:
rows = {'1' : [1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]}

In [13]:
state = (1,1,1,1,1,2,2,3,5,11,2,3,4,5,2,1,4)

In [14]:
def determine_reward():
    if referee(action_space,full_state)[0] == False:
        return -1
    elif game_over() == True:
        return 1
    else:
        return 0

In [15]:
def learning(state,new_state,action,lr,reward,gamma):#state and action are int taken from dictionaries
    Q[state,action] = Q[state,action] + lr * (reward + gamma * np.max(Q[new_state,:] - Q[state,action]))

In [16]:
def game_over(board):
    negative_counter = 0
    positive_counter = 0
    for x in action_space:
        if referee(x)[0] == True:###CHECK if ref takes in consideration if it's W or B turn.
            return False
    for x in board:
        if x < 0:
            negative_counter += 1
        elif x > 0:
            positive_counter += 1
    if negative_counter > 0 and positive_counter > 0:
        return False
    return True
    

In [17]:
def append_dict(state,Q):
    if state not in rows.values():
        rows[f'{len(rows)+1}'] = state
        Q = np.append(Q,zeros,axis = 0)
        return Q
    return Q

In [18]:
Q = append_dict([1,2,3,4,5,6,7,8,9],Q)

In [19]:
rows

{'1': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1],
 '2': [1, 2, 3, 4, 5, 6, 7, 8, 9]}

In [20]:
Q.shape

(2, 98)

In [21]:
rows

{'1': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1,
  -1],
 '2': [1, 2, 3, 4, 5, 6, 7, 8, 9]}

In [22]:
def get_temporal_difference(state_index,move_index,reward,gamma):
    max_action = np.argmax(Q[state_index])
    max_Q = Q[state_index][max_action]
    current_Q = Q[state_index][move_index]
    return reward + (gamma * (max_Q - current_Q))

# training

In [23]:
lr = 1
gamma = 0.8
reward = 0

In [24]:
full_state = [[0,1,1,1,1],1,(4,10)]

In [25]:
state = full_state[0]

In [26]:
def epsilon_greed(current_board,epsilon):
    state_number = 0
    for number,board_state in rows.items():
        if board == current_board:
            state_number = number
        else:
            append_dict(full_state[0])
    if np.random.random() < epsilon:
        return np.argmax(Q[state_number])
    else:
        return np.random.randint(98)

In [33]:
full_state = []

In [28]:
np.argmax(Q[1])

0

In [29]:
Q[1][19] = 23

In [30]:
def get_move(index):
    return action_space[index]

In [35]:
for episode in range(1000):
    board = full_state[0]
    state_index = 0
    for number,board_state in rows.items():
        if board == full_state[0]:
            state_index = number
        else:
            append_dict(full_state[0])
            for number,board_state in rows.items():
                if board == full_state[0]:
                    state_index = number
            
    while game_over(board) == False:
        move_index = epsilon_greed(full_state,epsilon)
        move = get_move(move_index)
        legal_boolean = referee(full_state) #referee the move
        if legal_boolean == False:
            reward = -1
            temporal_difference = get_temporal_difference
            Q[state_index][move_index] = Q[state_index][move_index] + lr * temporal_difference
            continue 
        reward = get_reward()
        nextState()
        Q[state_index][move_index] = Q[state_index][move_index] + lr * temporal_difference

NameError: name 'referee' is not defined